In [ ]:
!pip install gdown 

In [ ]:
!gdown --id 17u-Ty4j7oyr3-OPNtY5avgCmNWEJbISF --output ML.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=17u-Ty4j7oyr3-OPNtY5avgCmNWEJbISF
To: /content/ML.zip
100% 46.9M/46.9M [00:00<00:00, 127MB/s]


In [ ]:
!unzip /content/ML.zip

Archive:  /content/ML.zip
   creating: MovieLens-KG/
  inflating: MovieLens-KG/hop3.kg    
  inflating: MovieLens-KG/relation.kg  
  inflating: MovieLens-KG/link.kg    
  inflating: MovieLens-KG/hop1.kg    
  inflating: MovieLens-KG/relation_full.kg  
  inflating: MovieLens-KG/hop2.kg    
  inflating: MovieLens-KG/extra.kg   


In [ ]:
import os
os.chdir('/content')

In [ ]:
!git clone -b Inductive_learning https://@github.com/tinkoff-ai/kg-recsys.git

Cloning into 'kg-recsys'...
remote: Enumerating objects: 1495, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 1495 (delta 65), reused 80 (delta 31), pack-reused 1374
Receiving objects: 100% (1495/1495), 83.90 MiB | 20.25 MiB/s, done.
Resolving deltas: 100% (407/407), done.
Checking out files: 100% (140/140), done.


In [ ]:
import pandas as pd
import re
directory = '/content/kg-recsys/PinSage/data'
ratings = []
with open(os.path.join(directory, 'ratings.dat'), encoding='latin1') as f:
    for l in f:
        user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
        ratings.append({
            'user_id': user_id,
            'movie_id': movie_id,
            'rating': rating,
            'timestamp': timestamp,
            })
ratings = pd.DataFrame(ratings)

In [ ]:
!pip install dask

In [ ]:
!python -m pip install 'fsspec>=0.3.3'

     |████████████████████████████████| 136 kB 24.6 MB/s 


In [ ]:
!pip install partd

In [ ]:
import numpy as np
import dask.dataframe as dd
import pandas as pd
import pickle
import scipy.sparse as ssp

In [ ]:
def train_test_split_by_time(df, timestamp, user):
    df['train_mask'] = np.ones((len(df),), dtype=np.bool)
    df['val_mask'] = np.zeros((len(df),), dtype=np.bool)
    df['test_mask'] = np.zeros((len(df),), dtype=np.bool)
    df = dd.from_pandas(df, npartitions=10)
    def train_test_split(df):
        df = df.sort_values([timestamp])
        ##
        n_inter_items = df.shape[0]
        n_train_items = int(n_inter_items * 0.7)
        n_test_items = int(n_inter_items * 0.2)
        ##
        if df.shape[0] > 1:
            # df.iloc[:int(n_inter_items * 0.5), -3] = False
            df.iloc[-n_test_items:, -3] = False
            df.iloc[-n_test_items:, -1] = True
        if df.shape[0] > 2:
            df.iloc[n_train_items:-n_test_items, -3] = False
            df.iloc[n_train_items:-n_test_items, -2] = True
        return df
    df = df.groupby(user, group_keys=False).apply(train_test_split).compute(scheduler='processes').sort_index()
    # print(df[df[user] == df[user].unique()[0]].sort_values(timestamp))
    return df['train_mask'].to_numpy().nonzero()[0], \
           df['val_mask'].to_numpy().nonzero()[0], \
           df['test_mask'].to_numpy().nonzero()[0]

In [ ]:
train_indices, val_indices, test_indices = train_test_split_by_time(ratings, 'timestamp', 'user_id')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_

In [ ]:
df_train  = ratings.iloc[train_indices, :].copy()
df_test  = ratings.iloc[test_indices, :].copy()
df_val  = ratings.iloc[val_indices, :].copy()

In [ ]:
df_train['user_id'].nunique(), df_test['user_id'].nunique(), df_val['user_id'].nunique()

(6040, 6040, 6040)

In [ ]:
df_train.shape, df_test.shape, df_val.shape

((697378, 7), (197656, 7), (105175, 7))

In [ ]:
697378+197656+105175

1000209

In [ ]:
df_train.iloc[:, :4].to_csv('df_train.csv', index = False)
df_test.iloc[:, :4].to_csv('df_test.csv', index = False)
df_val.iloc[:, :4].to_csv('df_val.csv', index = False)

In [ ]:
users = []
with open('/content/MovieLens-KG/link.txt', encoding='latin1') as f:
    for l in f:
        # print(l)
        id_2, id_1= l.strip().split('	')
        users.append({
            'id_2': (id_2),
            'id_1': (id_1)            })
users = pd.DataFrame(users).astype('category')

In [ ]:
import numpy as np
len(np.intersect1d(np.array(users['id_2'].astype('int32').unique()), np.array(ratings['movie_id'].unique())))

3655

In [ ]:
ratings['movie_id'].nunique()

3706

In [ ]:
filenames = ['/content/MovieLens-KG/hop1.kg', '/content/MovieLens-KG/hop2.kg', '/content/MovieLens-KG/hop3.kg']
with open('/content/kg.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

Train test split

In [ ]:
!git clone https://github.com/hwwang55/KGCN.git

Cloning into 'KGCN'...
remote: Enumerating objects: 271, done.
remote: Total 271 (delta 0), reused 0 (delta 0), pack-reused 271
Receiving objects: 100% (271/271), 175.19 MiB | 22.27 MiB/s, done.
Resolving deltas: 100% (143/143), done.


In [ ]:
import os
os.chdir('/content/KGCN/data/movie')

In [ ]:
!gdown --id 1cVnS454hFvQ_FFzEMGoIp9Yi8tYAb5st --output kg_final.txt

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1cVnS454hFvQ_FFzEMGoIp9Yi8tYAb5st
To: /content/KGCN/data/movie/kg_final.txt
100% 67.9M/67.9M [00:00<00:00, 102MB/s]


In [ ]:
!gdown --id 1cfXoRRGTALlY460EEhSKKPEloS_bbbdZ --output kg.txt

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1cfXoRRGTALlY460EEhSKKPEloS_bbbdZ
To: /content/KGCN/data/movie/kg.txt
100% 192M/192M [00:00<00:00, 254MB/s]


In [ ]:
!gdown --id 17r3YnU9GGIji06OiHPDLhPgzInXzkkiF --output ratings_final.txt

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=17r3YnU9GGIji06OiHPDLhPgzInXzkkiF
To: /content/KGCN/data/movie/ratings_final.txt
100% 23.0M/23.0M [00:00<00:00, 150MB/s] 


In [ ]:
!gdown --id 1Hwho5dOYF4DezRwX_9o39jpeYfDAyPhO --output ratings_final_train.txt

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Hwho5dOYF4DezRwX_9o39jpeYfDAyPhO
To: /content/KGCN/data/movie/ratings_final_train.txt
100% 16.0M/16.0M [00:00<00:00, 92.2MB/s]


In [ ]:
!gdown --id 19DuoWnxZpMiFPPtqZVto88wvlTG-0-dc --output ratings_final_test.txt

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=19DuoWnxZpMiFPPtqZVto88wvlTG-0-dc
To: /content/KGCN/data/movie/ratings_final_test.txt
100% 4.55M/4.55M [00:00<00:00, 51.5MB/s]


In [ ]:
!gdown --id 14NdCngBDANOapbk5kod-RKjfCFATYWB- --output ratings_final_val.txt

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=14NdCngBDANOapbk5kod-RKjfCFATYWB-
To: /content/KGCN/data/movie/ratings_final_val.txt
100% 2.42M/2.42M [00:00<00:00, 169MB/s]


In [ ]:
import os
os.chdir('/content/KGCN/src')

In [ ]:
ratings.user_id.nunique()

6040

In [ ]:
import numpy as np
cheat_array = np.full((3, 5), 7, dtype=int)

In [ ]:
cheat_array[1, 1] =  

array([7, 7, 7, 7, 7])

In [130]:
import pickle
with open('/content/KGCN/src/pos_samples_NEW.pkl', 'rb') as f:
    cheat2 = pickle.load(f)

In [ ]:
import pickle
with open('/content/KGCN/data/movie/ml_kgcn_mapping.pkl_', 'rb') as f:
    mapping_ = pickle.load(f)

In [ ]:
mapping_['2600']

2364

{1283, 2185, 1173, 1432, 3497, 1452, 2349, 49, 1459, 3509, 441, 827, 571, 828, 575, 1860, 836, 1735, 589, 1749, 2518, 3416, 1249, 2788, 2916, 998, 744, 1001, 2667, 366, 1141, 1525, 1273, 3197, 511}

In [ ]:
import pandas as pd
df_= pd.read_csv('/content/KGCN/data/movie/ratings.csv')

In [ ]:
cheat[2577][np.where(cheat[2577]!=7000)]

array([  10,  165,  276,  308,  501,  546,  633,  714,  771,  805, 1061,
       1159, 1285, 1358, 1411, 1488, 1836, 1866, 2016, 2063, 2097, 2131,
       2176, 2208, 2306, 2584, 2609, 2637, 2792, 2817, 2969, 3070, 3224,
       3433])

In [131]:
cheat2[2577][np.where(cheat2[2577]!=7000)]

array([  10,  165,  276,  501,  714,  771, 1159, 1285, 1358, 1488, 1836,
       1866, 2016, 2063, 2097, 2131, 2176, 2208, 2609, 2792, 2817, 3070,
       3224, 3433])

In [ ]:
new_i = []
for i in df_[df_['user_id']==2578].movie_id.values:
  new_i.append(mapping_[str(i)])

In [ ]:
np.sort(np.array(new_i))

array([  49,  366,  441,  511,  571,  575,  589,  744,  827,  828,  836,
        998, 1001, 1141, 1173, 1249, 1273, 1283, 1432, 1452, 1459, 1525,
       1735, 1749, 1860, 2185, 2349, 2518, 2667, 2788, 2916, 3197, 3416,
       3497, 3509])

In [ ]:
import pandas as pd
df_ = pd.read_csv('/content/KGCN/data/movie/ratings.csv')

In [129]:
!python preprocess.py -d movie

reading item index to entity id file: ../data/movie/item_index2entity_id.txt ...
item_index_old2new 2752 2364
reading rating file ...
converting rating file ...
number of users: 6040
number of items: 3655
user_index_old2new 22 2577 5677
converting kg file ...
number of entities (containing items): 774885
number of relations: 44
done


In [ ]:
 1215+1919

3134

In [ ]:
!pip install pip install tensorflow==1.15.5

     |████████████████████████████████| 110.5 MB 1.4 kB/s 
     |████████████████████████████████| 2.9 MB 68.8 MB/s 
     |████████████████████████████████| 50 kB 4.4 MB/s 
     |████████████████████████████████| 503 kB 79.5 MB/s 
     |████████████████████████████████| 3.8 MB 4.5 MB/s 
     |████████████████████████████████| 20.1 MB 84.8 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=9b956af1f9def45d8422fd7ffcc167b95774b0275b83d6aad3b307369a8e5594
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tens

In [ ]:
precision: 0.0631	0.0651	0.0608	0.0554	0.0498	
recall: 0.0043	0.0086	0.0200	0.0369	0.0657	

In [144]:
!python main.py

reading rating file ...
reading KG file ...
constructing knowledge graph ...
constructing adjacency matrix ...
data loaded.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2022-05-15 18:37:11.559623: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-05-15 18:37:11.563326: I tensorflow/core/platform/profil

In [145]:
import pickle
with open('/content/KGCN/src/KGCN_preds_for_ml.pkl', 'rb') as f:
    res = pickle.load(f)

In [146]:
import numpy as np
res = np.array(res)

In [147]:
import numpy as np
import torch
import pickle



def read_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        lines = f.read().strip().split('\n')
    for line in lines:
        items = line.split(' ')[1:]
        items = [int(item) for item in items]
        # if items:
        #     self.n_items = max(self.n_items, max(items) + 1)
        data.append(items)
    return data

def calculate_metrics(eval_data, rec_items, topks):
    results = {'Precision': {}, 'Recall': {}, 'NDCG': {}}
    hit_matrix = np.zeros_like(rec_items, dtype=np.float32)
    for user in range(rec_items.shape[0]):
        for item_idx in range(rec_items.shape[1]):
            if rec_items[user, item_idx] in eval_data[user]:
                hit_matrix[user, item_idx] = 1.
    eval_data_len = np.array([len(items) for items in eval_data], dtype=np.int32)

    for k in topks:
        hit_num = np.sum(hit_matrix[:, :k], axis=1)
        precisions = hit_num / k
        with np.errstate(invalid='ignore'):
            recalls = hit_num / eval_data_len

        max_hit_num = np.minimum(eval_data_len, k)
        max_hit_matrix = np.zeros_like(hit_matrix[:, :k], dtype=np.float32)
        for user, num in enumerate(max_hit_num):
            max_hit_matrix[user, :num] = 1.
        denominator = np.log2(np.arange(2, k + 2, dtype=np.float32))[None, :]
        dcgs = np.sum(hit_matrix[:, :k] / denominator, axis=1)
        idcgs = np.sum(max_hit_matrix / denominator, axis=1)
        with np.errstate(invalid='ignore'):
            ndcgs = dcgs / idcgs

        user_masks = (max_hit_num > 0)
        results['Precision'][k] = precisions[user_masks].mean()
        results['Recall'][k] = recalls[user_masks].mean()
        results['NDCG'][k] = ndcgs[user_masks].mean()
    return results


def eval(recommendations, groud_truth_path, top_k):
        eval_data = read_data(groud_truth_path)
        metrics = calculate_metrics(eval_data, recommendations, top_k)
        precison = ''
        recall = ''
        ndcg = ''
        for k in top_k:
            precison += '{:.3f}%@{:d}, '.format(metrics['Precision'][k] * 100., k)
            recall += '{:.3f}%@{:d}, '.format(metrics['Recall'][k] * 100., k)
            ndcg += '{:.3f}%@{:d}, '.format(metrics['NDCG'][k] * 100., k)
        results = 'Precision: {:s}Recall: {:s}NDCG: {:s}'.format(precison, recall, ndcg)
        return results, metrics






In [148]:
# # movie
# parser.add_argument('--dataset', type=str, default='movie', help='which dataset to use')
# parser.add_argument('--aggregator', type=str, default='sum', help='which aggregator to use')
# parser.add_argument('--n_epochs', type=int, default=100, help='the number of epochs')
# parser.add_argument('--neighbor_sample_size', type=int, default=3, help='the number of neighbors to be sampled')
# parser.add_argument('--dim', type=int, default=32, help='dimension of user and entity embeddings')
# parser.add_argument('--n_iter', type=int, default=2, help='number of iterations when computing entity representation')
# parser.add_argument('--batch_size', type=int, default=256, help='batch size')
# parser.add_argument('--l2_weight', type=float, default=1e-5, help='weight of l2 regularization')
# parser.add_argument('--lr', type=float, default=2e-5, help='learning rate')
# parser.add_argument('--ratio', type=float, default=1, help='size of training dataset')

In [149]:
eval(res, '/content/KGCN/data/movie/test.txt', [1, 5, 10, 15, 20])

('Precision: 10.199%@1, 10.116%@5, 9.639%@10, 9.238%@15, 8.845%@20, Recall: 0.375%@1, 2.003%@5, 3.784%@10, 5.308%@15, 6.905%@20, NDCG: 10.199%@1, 10.198%@5, 10.125%@10, 10.200%@15, 10.379%@20, ',
 {'NDCG': {1: 0.10198676,
   5: 0.10197909,
   10: 0.10125298,
   15: 0.10199534,
   20: 0.10379004},
  'Precision': {1: 0.10198676,
   5: 0.10115894,
   10: 0.09639072,
   15: 0.09238411,
   20: 0.08845199},
  'Recall': {1: 0.0037504605019015465,
   5: 0.02002631399768271,
   10: 0.03784400304649605,
   15: 0.05307867829051496,
   20: 0.06905104489776645}})

In [138]:
eval(res, '/content/KGCN/data/movie/test.txt', [1, 5, 10, 15, 20])

('Precision: 10.828%@1, 9.911%@5, 9.518%@10, 9.017%@15, 8.613%@20, Recall: 0.473%@1, 1.909%@5, 3.660%@10, 5.162%@15, 6.588%@20, NDCG: 10.828%@1, 10.134%@5, 10.084%@10, 10.088%@15, 10.205%@20, ',
 {'NDCG': {1: 0.10827815,
   5: 0.101344585,
   10: 0.10083604,
   15: 0.100880906,
   20: 0.10204594},
  'Precision': {1: 0.10827815,
   5: 0.099105954,
   10: 0.09518212,
   15: 0.09016557,
   20: 0.086134106},
  'Recall': {1: 0.0047305788877774636,
   5: 0.019093036839281617,
   10: 0.036600439619829214,
   15: 0.05161952018688298,
   20: 0.06588044071484414}})

In [ ]:
eval(res, '/content/KGCN/data/movie/test.txt', [1, 5, 10, 15, 20])

('Precision: 11.308%@1, 10.089%@5, 9.349%@10, 8.959%@15, 8.543%@20, Recall: 0.454%@1, 1.981%@5, 3.655%@10, 5.225%@15, 6.624%@20, NDCG: 11.308%@1, 10.365%@5, 10.043%@10, 10.091%@15, 10.198%@20, ',
 {'NDCG': {1: 0.11307947,
   5: 0.10364631,
   10: 0.10043246,
   15: 0.100906454,
   20: 0.10198354},
  'Precision': {1: 0.11307947,
   5: 0.10089404,
   10: 0.09349338,
   15: 0.089591615,
   20: 0.085430466},
  'Recall': {1: 0.004540846692851485,
   5: 0.019809890961214542,
   10: 0.03655426172112688,
   15: 0.05225392250313733,
   20: 0.06624145407794223}})

In [ ]:
eval(res, '/content/KGCN/data/movie/test.txt', [1, 5, 10, 15, 20])

('Precision: 10.530%@1, 9.851%@5, 9.207%@10, 8.717%@15, 8.402%@20, Recall: 0.421%@1, 2.012%@5, 3.656%@10, 5.062%@15, 6.484%@20, NDCG: 10.530%@1, 10.076%@5, 9.846%@10, 9.825%@15, 9.996%@20, ',
 {'NDCG': {1: 0.10529801,
   5: 0.10076411,
   10: 0.09845884,
   15: 0.098253265,
   20: 0.09996467},
  'Precision': {1: 0.10529801,
   5: 0.09850993,
   10: 0.09206953,
   15: 0.08717439,
   20: 0.08402318},
  'Recall': {1: 0.0042050863226847195,
   5: 0.020119720531003207,
   10: 0.036560151952862815,
   15: 0.050620565395781504,
   20: 0.06483800436707379}})

parser.add_argument('--dataset', type=str, default='movie', help='which dataset to use')
parser.add_argument('--aggregator', type=str, default='sum', help='which aggregator to use')
parser.add_argument('--n_epochs', type=int, default=100, help='the number of epochs')
parser.add_argument('--neighbor_sample_size', type=int, default=4, help='the number of neighbors to be sampled')
parser.add_argument('--dim', type=int, default=32, help='dimension of user and entity embeddings')
parser.add_argument('--n_iter', type=int, default=2, help='number of iterations when computing entity representation')
parser.add_argument('--batch_size', type=int, default=560, help='batch size')
parser.add_argument('--l2_weight', type=float, default=1e-5, help='weight of l2 regularization')
parser.add_argument('--lr', type=float, default=2e-4, help='learning rate')
parser.add_argument('--ratio', type=float, default=1, help='size of training dataset')


In [ ]:
eval(res, '/content/KGCN/data/movie/test.txt', [1, 5, 10, 15, 20])

('Precision: 4.089%@1, 4.046%@5, 4.250%@10, 4.425%@15, 4.286%@20, Recall: 0.232%@1, 1.086%@5, 2.298%@10, 3.688%@15, 4.706%@20, NDCG: 4.089%@1, 4.103%@5, 4.447%@10, 4.908%@15, 5.189%@20, ',
 {'NDCG': {1: 0.04089404,
   5: 0.041029803,
   10: 0.0444673,
   15: 0.04907563,
   20: 0.0518854},
  'Precision': {1: 0.04089404,
   5: 0.040463578,
   10: 0.0425,
   15: 0.04424945,
   20: 0.04286424},
  'Recall': {1: 0.0023181524209773736,
   5: 0.010855954451943496,
   10: 0.02297697040065294,
   15: 0.036878267642961524,
   20: 0.04705563880375622}})

Параметры:
# movie
parser.add_argument('--dataset', type=str, default='movie', help='which dataset to use')
parser.add_argument('--aggregator', type=str, default='sum', help='which aggregator to use')
parser.add_argument('--n_epochs', type=int, default=100, help='the number of epochs')
parser.add_argument('--neighbor_sample_size', type=int, default=4, help='the number of neighbors to be sampled')
parser.add_argument('--dim', type=int, default=32, help='dimension of user and entity embeddings')
parser.add_argument('--n_iter', type=int, default=2, help='number of iterations when computing entity representation')
parser.add_argument('--batch_size', type=int, default=2056, help='batch size')
parser.add_argument('--l2_weight', type=float, default=1e-5, help='weight of l2 regularization')
parser.add_argument('--lr', type=float, default=2e-5, help='learning rate')
parser.add_argument('--ratio', type=float, default=1, help='size of training dataset')


In [ ]:
eval(res, '/content/KGCN/data/movie/test.txt', [1, 5, 10, 15, 20])

('Precision: 4.089%@1, 4.046%@5, 4.093%@10, 4.137%@15, 4.135%@20, Recall: 0.232%@1, 1.147%@5, 2.260%@10, 3.458%@15, 4.474%@20, NDCG: 4.089%@1, 4.133%@5, 4.349%@10, 4.697%@15, 5.037%@20, ',
 {'NDCG': {1: 0.04089404,
   5: 0.04132524,
   10: 0.043492004,
   15: 0.046969898,
   20: 0.050371543},
  'Precision': {1: 0.04089404,
   5: 0.040463578,
   10: 0.040927153,
   15: 0.041368652,
   20: 0.041349337},
  'Recall': {1: 0.0023181524209773736,
   5: 0.011468877211355654,
   10: 0.022600285563828974,
   15: 0.03458228439066443,
   20: 0.04474072442236445}})

По шагам:
- написать раннюю остановку - ok
- метрики оценивать на валидации -ok
- изменить батч сайз - ok
- переобучить после ранней остановки 
- сделать метод предикт, выдающий для всех пользователей все предсказания - ok
- достать из препроцессинга мэппиг юзеров и айтемов - ok
- выходной набор данных перемэппить - ok
- сверить что gt сходятся с ремаппингом